In [48]:
from pathlib import Path
import sys
import pandas as pd
# ensure project root is on sys.path so `scripts` is importable
sys.path.append(str(Path('..').resolve()))
from scripts.image_to_csv import process_dataset
from scripts.split_data import split_csv_streaming
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [25]:
dataset_path = '../assets/image_dataset/tomato_objects_hog.csv'
# process_dataset(dataset_path) # if you want to run it again remove the comment
# the comment added to prevent re-processing the dataset since it takes alot of time

In [ ]:
df = pd.read_csv('../assets/image_dataset/tomato_objects_hog.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63956 entries, 0 to 63955
Columns: 1765 entries, f0 to label
dtypes: float64(1764), int64(1)
memory usage: 861.2 MB


In [16]:
df.isnull().all().sum()

np.int64(0)

In [ ]:
split_csv_streaming()

In [49]:
# Loading the CSV into a DataFrame
train_df = pd.read_csv('../assets/image_dataset/train_data.csv')

In [50]:
# Separate Features (X) and Labels (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

In [51]:
test_df = pd.read_csv('../assets/image_dataset/test_data.csv')
    
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

In [52]:
# max_iter=3000 ensures the model has enough time to find the best math
# solver='lbfgs' is the standard, efficient solver
model = LogisticRegression(max_iter=3000, solver='lbfgs', verbose=1)
    
model.fit(X_train, y_train)
print("   Training complete.")

   Training complete.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.9s finished


In [53]:
y_pred = model.predict(X_test)

In [54]:
# Calculate Accuracy
acc = accuracy_score(y_test, y_pred)

print(f"FINAL ACCURACY: {acc * 100:.2f}%")

FINAL ACCURACY: 78.70%


In [57]:
# clasification report
report = classification_report(y_test, y_pred, target_names=['Fresh (0)', 'Rotten (1)'])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

   Fresh (0)       0.20      0.00      0.00      2722
  Rotten (1)       0.79      1.00      0.88     10074

    accuracy                           0.79     12796
   macro avg       0.49      0.50      0.44     12796
weighted avg       0.66      0.79      0.69     12796

